In [ ]:
#encoding:utf-8
###########import packages##########
import shap
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost 
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('datatest.csv',encoding="gbk")
raw_data=fdata.loc[:,['Dielectric Constant (C  N-1 M-2)',#0
                      'Flash Point(℃)',#1
                      'Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)',#2
                      'Flow Rate (mL min-1)',#3
                      'Water Content (wt%)',#4
                      'Anodic Platinum Loading Amount (mgPt cm-2)',#5
                      'Cathodic Platinum Loading Amount (mgPt cm-2)',#6
                      'I/C',#7
                      'Cell Temperature (℃)',#8
                      'Anode Flow Rate (sccm)',#9
                      'Cathode Flow Rate (sccm)',#10
                      'Solid Content (wt%)',#11
                      'Backpressure (Mpa)',#12
                      'Pt Consumption per kW@0.65V (mgpt kW-1)'#13
                        ]]



###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=raw_data.iloc[:,0:13]
raw_output=raw_data.iloc[:,13]
# raw_input_global=raw_data.iloc[:,0:32]
# raw_output_global=raw_data.iloc[:,32]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})


In [ ]:
def gridsearch(model,param,algorithm_name):
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test.values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    
    result_train = best_model.predict(X_train)
    x_prediction_07_train=result_train
    y_real_07_train=y_train.values
    x_prediction_07_series_train=pd.Series(x_prediction_07_train)
    y_real_07_series_train=pd.Series(y_real_07_train)
    
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    
    corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
    error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
    
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val,'TEST R2',error_val[3],'TEST CORR',corr_ann)
    print(error_val_train,'TRAIN R2',error_val_train[3],'TRAIN CORR',corr_ann_train)
    x_y_x=np.arange(0,2.1,0.1)
    x_y_y=np.arange(0,2.1,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name+' Test Set',alpha=0.75)
    ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label=algorithm_name+' Training Set',alpha=0.25,marker="^")
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Pt_Consumption_per_kW@0.65V (mgpt kW-1)")
    plt.ylabel(u"Real_Pt_Consumption_per_kW@0.65V (mgpt kW-1)")
    print('finished')

In [ ]:
def shap_plot(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:13]
    SHAP_OUTPUT=raw_data.iloc[:,13]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    cols=[                                   
                           'Membrane Thickness (μm)',#0
                      'GDE(1 for GDE 0 for CCM)',#1
                      'Dielectric Constant (C  N-1 M-2)',#2
                      'Flash Point(℃)',#3
                      'Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)',#4
                      'Stirring Temp (℃)',#5
                      'Flow Rate (mL min-1)',#6
                      'Water Content (wt%)',#7
                      'Nafion EW (mol g-1)',#8
                      'Anodic Platinum Loading Amount (mgPt cm-2)',#9
                      'Cathodic Platinum Loading Amount (mgPt cm-2)',#10
                      'Drying Temperature (℃)',#11
                      'I/C',#12
                      'Cell Temperature (℃)',#13
                      'Anode Flow Rate (sccm)',#14
                      'Cathode Flow Rate (sccm)',#15
                      'Active Area (cm2)',#16
                      'Solid Content (wt%)',#17
                      'Backpressure (Mpa)'
                            ]
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
#         shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        shap.dependence_plot('Water Content (wt%)', shap_values, SHAP_INPUT, interaction_index=None, show=False)

        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)

    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
#         shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        shap.dependence_plot('Water Content (wt%)', shap_values, SHAP_INPUT, interaction_index=None, show=False)

        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)

In [ ]:
seed=17
X_train, X_test, y_train, y_test = train_test_split(raw_input, raw_output, test_size=.15,random_state=seed)
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt','rf'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'n_estimators':[50,100,200,400],
'max_depth':[5,7,9,11,13,-1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'max_depth':[5,7,9,11,13],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'max_depth':[5,7,9,11],
'boosting_type':['plain'],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'criterion':['friedman_mse','mae','mse'],
'max_features':['auto','sqrt','log2'],
'loss':['ls', 'lad', 'huber', 'quantile'],
'max_depth':[3,5,7,9,11,16]
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'criterion':['mse','mae'],
'max_features':['auto','sqrt','log2'],
'n_estimators':[10,50,100,200,400],
'max_depth':[3,5,7,9,11,None]
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
'max_depth':[5,6,7,8,9,10,11,None],
'max_features':['auto','sqrt','log2'],
'criterion' : ["mse", "friedman_mse", "mae"],
'splitter' : [ "best",'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
'max_depth':[5,6,7,8,9,10,11,None],
'max_features':['auto','sqrt','log2'],
'criterion' : ["mse", "friedman_mse", "mae"],
'splitter' : [ "best",'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'loss':['linear', 'square', 'exponential']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')

In [ ]:
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
 'boosting_type':['gbdt'],
    'learning_rate':[0.1],
    'subsample':[0.4],
     'n_estimators':[100],
    'max_depth':[5],
 'reg_alpha':[0.01],
 'reg_lambda':[0.001]
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

In [ ]:
shap_plot(model_LightGBMRegressor,param_light,'LightGBM')

In [ ]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.14],
 'n_estimators':[200],
 'max_depth':[7],
 'subsample':[0.4],
 'reg_alpha':[0],
 'reg_lambda':[1e-05]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

In [ ]:
shap_plot(model_XGBRegressor,param_xg,'XGBoost')

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.16],
 'n_estimators':[50],
 'max_depth':[5],
 'subsample':[0.4],
    'reg_lambda':[0.0001]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')

In [ ]:
from catboost import *
shap_plot(model_CatRegressor,param_cat,'CatBoost')

In [ ]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.2],
                'criterion':['mae'],
                 'max_features':['sqrt'],
                 'loss':['ls'],
    'max_depth':[3]
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

In [ ]:
shap_plot(model_GradientBoostingRegressor,param_GB,'GradientBoost')

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
            'criterion':['mse'],
            'max_features':['sqrt'],
    'n_estimators':[10],
     'max_depth':[None]
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')

In [ ]:
shap_plot(model_RandomForestRegressor,param_RF,'Random Forest')

In [ ]:
###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
         'max_depth':[10],
               'max_features':['sqrt'],
               'criterion' : [ "mae"],
               'splitter' : [ 'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')

In [ ]:
shap_plot(model_ExtraTreeRegressor,param_ET,'Extra Tree')

In [ ]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
        'max_depth':[10],
               'max_features':['sqrt'],
               'criterion' : [ "mae"],
               'splitter' : [ 'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')

In [ ]:
shap_plot(model_DecisionTreeRegressor,param_DT,'Decision Tree')

In [ ]:
###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
     'learning_rate':[1],
    'n_estimators':[400],
            'loss':['square']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')

In [ ]:
shap_plot(model_AdaBoostRegressor,param_Ada,'AdaBoost')